# Decision Tree

## 1. Implementation - using gini index

In [1]:
import numpy as np
import pandas as pd

In [2]:
# group is a iterator of np.array of integers, where each integer represents a class type
# gini_indx: np.array --> float

def gini_index(groups):
    # count all samples at split point
    n_instances = sum([len(group) for group in groups])
    
    # sum weighted Gini index for each group
    gini = 1.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue

        # score the group based on the score for each class
        unique, counts = np.unique(group, return_counts=True)
        #print(counts)
        score = np.sum((counts / size) ** 2)
 
        # weight the group score by its relative size
        gini -= score * (size / n_instances)
        
    return gini
 
# test Gini values
# print(gini_index(np.array([[1, 0], [1, 0, 1]])))
# print(gini_index(np.array([[0, 0], [1, 1, 1]])))

def try_split(index, value, data):
    feature = data[:, index]
    
    left_index = np.where(feature < value)[0]
    right_index = np.where(feature >= value)[0]
    
    return left_index, right_index       

def get_binary_split(data, labels):
    b_index, b_gini, b_value, b_left, b_right = np.nan, 1, np.nan, None, None
    
    num_features = len(data[0])
    
    for index in range(0, num_features):
        for row in data:
            left_index, right_index = try_split(index, row[index], data)
            gini = gini_index([labels[left_index], labels[right_index]])
            
            if gini < b_gini:
                b_index, b_gini, b_value, b_left, b_right = index, gini, row[index], left_index, right_index
    
    return {'gini': b_gini, 'index':b_index, 'value':b_value, 'left':b_left, 'right':b_right}  

class DecisionTree(object):
    
    def __init__(self, depth=1):
        self.depth = depth
        self.node = None
        self.left = None
        self.right = None
        
    def most_freq(self, arr):
        unique, counts = np.unique(arr, return_counts=True)
        ind = np.argmax(counts)
        return unique[ind]
    
    # data is an np.array of size N times D
    # N = number of samples
    # D = number of features
    # labels is an np.array of size N times 1
    def train(self, data, labels):
        if self.depth == 0:
            self.node = (self.most_freq(labels), 0, None, None)
        else:
            splited_data = get_binary_split(data, labels)
            
            #print(len(splited_data['left']))
            #print(len(splited_data['right']))
            
            if (len(splited_data['left']) == 0) or (len(splited_data['right']) ==0):
                self.depth = 0
                self.node = (self.most_freq(labels), 0, None, None)
            else:
                self.node = (None, splited_data['gini'], splited_data['index'], splited_data['value'])
                self.left = DecisionTree(self.depth-1)
                self.left.train(data[splited_data['left']], labels[splited_data['left']])
                
                self.right = DecisionTree(self.depth-1)
                self.right.train(data[splited_data['right']], labels[splited_data['right']])  
                
                
        
                

    # row is a np.array of size D
    def predict_single(self, row):
        if self.depth == 0:
            return self.node[0]
        else:
            index = self.node[2]
            value = self.node[-1]

            if row[index] < value:
                return self.left.predict_single(row)
            else:
                return self.right.predict_single(row)
                    
    def predict(self, X):
        ypred = []
        
        for row in X:
            ypred.append(self.predict_single(row))
        
        return ypred
    
    def show_structure(self, max_depth=-1):
        if max_depth == -1:
            max_depth = self.depth
            
        if self.depth > 0:
            tabs = '\t' * (max_depth-self.depth)
            m = '{}[Depth {}: index={}, value={}, gini={}]'.format(tabs,
                                                                   max_depth-self.depth+1, 
                                                                   self.node[2],
                                                                   self.node[3],
                                                                   round(self.node[1],3))
            print(m)
            if self.left != None:
                self.left.show_structure(max_depth)
            if self.right != None:
                self.right.show_structure(max_depth)
        
        

# Bank Notes
Deciding if a bank note is fradulent

In [152]:
# Load the data
# data is first downloweded into DATA_PATH from 
# http://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt
import os

DATA_PATH = 'banknote'
FILE_NAME = 'data_banknote_authentication.txt'

def load_data(data_path=DATA_PATH, file_name=FILE_NAME):
    file_path = os.path.join(data_path, file_name)
    data = pd.read_csv(file_path)
    
    return data.values[:, :-1], data.values[:,-1]

DATA, LABELS = load_data()
LABELS = LABELS * 1.0

train_set = DATA[:1234, :]
train_labels = LABELS[:1234]

test_set = DATA[1234:,:]
test_labels = LABELS[1234:]

In [153]:
bank_auth = DecisionTree(4)

In [154]:
bank_auth.train(train_set, train_labels)

In [155]:
ypred = bank_auth.predict(test_set)
e = len(test_labels[test_labels != ypred])/len(test_labels)
e # error rate

0.0364963503649635

In [156]:
bank_auth.show_structure()

[Depth 1: index=0, value=0.3223, gini=0.246]
	[Depth 2: index=1, value=5.8974, gini=0.182]
		[Depth 3: index=2, value=6.2204, gini=0.108]
			[Depth 4: index=1, value=4.1158, gini=0.014]
			[Depth 4: index=1, value=-4.6062, gini=0.016]
		[Depth 3: index=0, value=-2.7419, gini=0.015]
			[Depth 4: index=2, value=3.1392, gini=0.0]
			[Depth 4: index=0, value=-1.8584, gini=-0.0]
	[Depth 2: index=2, value=-4.3839, gini=0.118]
		[Depth 3: index=0, value=4.2164, gini=0.0]
			[Depth 4: index=0, value=0.64376, gini=-0.0]
			[Depth 4: index=0, value=5.7456, gini=-0.0]
		[Depth 3: index=0, value=1.594, gini=0.09]
			[Depth 4: index=2, value=-2.2718, gini=0.175]
			[Depth 4: index=0, value=2.0421, gini=0.012]


# Iris

Classification of types of iris

In [157]:
from sklearn.datasets import load_iris

def prep_train_test(rate=0.9):
    iris = load_iris()
    X = iris.data
    y = iris.target
    ind = []
    
    for i in range(3):
        ind.append(np.random.choice(50, int(50*rate), replace=False) + 50*i)
    
    train_ind = np.concatenate(ind)
    test_ind = np.setdiff1d(np.arange(150), train_ind)
        
    
    return X[train_ind], y[train_ind], X[test_ind], y[test_ind]

iris_train_data, iris_train_labels, iris_test_data,iris_test_labels = prep_train_test(0.9)

In [158]:
iris_clf = DecisionTree(3)
iris_clf.train(iris_train_data, iris_train_labels)
iris_clf.show_structure()

[Depth 1: index=2, value=3.0, gini=0.333]
	[Depth 2: index=1, value=3.3, gini=-0.0]
		[Depth 3: index=0, value=4.8, gini=0.0]
		[Depth 3: index=1, value=3.6, gini=-0.0]
	[Depth 2: index=2, value=4.8, gini=0.103]
		[Depth 3: index=3, value=1.7, gini=0.0]
		[Depth 3: index=3, value=1.8, gini=0.103]


In [159]:
ypred = iris_clf.predict(iris_test_data)
e = len(iris_test_labels[iris_test_labels != ypred])/len(iris_test_labels)
e # error rate

0.06666666666666667